In [1]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.0

--2023-05-28 11:15:07--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/downloads/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2023-05-28 11:15:07--  https://developer.nvidia.com/downloads/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?ytleT_IvJDBrFFaUp9cAYpyXStS0AmMz5GkSrWiIMgxXiBN1EnDD8XIotFdWys-eoSUR3zV-392Mhz9-vVvfN7ISF

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [3]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-s155_0qx
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-s155_0qx
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=a040a1c3a880046bb383e3fee8b3df7187edba81a4243fdb7ada319c9b32a4ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-d_rjkkrs/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [6]:
%%cu
#include<stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>

 __global__ static void quicksort(int* values,int N) {
 #define MAX_LEVELS 300

 int pivot, L, R;
 int idx =  threadIdx.x + blockIdx.x * blockDim.x;
 int start[MAX_LEVELS];
 int end[MAX_LEVELS];

 start[idx] = idx;
 end[idx] = N - 1;
 while (idx >= 0) {
  L = start[idx];
  R = end[idx];
  if (L < R) {
   pivot = values[L];
   while (L < R) {
    while (values[R] >= pivot && L < R)
     R--;
    if(L < R)
     values[L++] = values[R];
    while (values[L] < pivot && L < R)
     L++;
    if (L < R)
     values[R--] = values[L];
   }
   values[L] = pivot;
   start[idx + 1] = L + 1;
   end[idx + 1] = end[idx];
   end[idx++] = L;
   if (end[idx] - start[idx] > end[idx - 1] - start[idx - 1]) {
                         // swap start[idx] and start[idx-1]
                         int tmp = start[idx];
                         start[idx] = start[idx - 1];
                         start[idx - 1] = tmp;

                         // swap end[idx] and end[idx-1]
                         tmp = end[idx];
                         end[idx] = end[idx - 1];
                         end[idx - 1] = tmp;
                 }

  }
  else
   idx--;
 }
}


int main(){
  int x[20],size,i;
  int *d_x,*d_size,*d_i;





  printf("Enter size of the array: ");
  scanf("%d",&size);

  printf("Enter %d elements: ",size);
  for(i=0;i<size;i++)
    scanf("%d",&x[i]);
  cudaMalloc((void **)&d_x,sizeof(int)*size);
    cudaMalloc((void **)&d_size,sizeof(int));
    cudaMalloc((void **)&d_i,sizeof(int));

  cudaMemcpy(d_x, &x,  sizeof( int)*size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_size, &size,  sizeof( int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_i, &i,  sizeof( int), cudaMemcpyHostToDevice);

  quicksort<<<1,1>>>(d_x,size);
  cudaMemcpy(x, d_x, sizeof(int)*size, cudaMemcpyDeviceToHost);
  printf("Sorted elements: ");
  for(i=0;i<size;i++)
    printf(" %d",x[i]);

  return 0;
}


Enter size of the array: Enter -1350720866 elements: Sorted elements: 
